In [1]:
import tensorflow as tf

tf.enable_eager_execution()

# import numpy as np
#
# # z1, y1, x1, z2, y2, x2
# a = np.array([0, 0, 1, 6, 7, 9])
# b = np.array([2, 5, 3, 10, 10, 10])
#
# zA = max(a[0], b[0])
# yA = max(a[1], b[1])
# xA = max(a[2], b[2])
# zB = max(a[3], b[3])
# yB = max(a[4], b[4])
# xB = max(a[5], b[5])
#
# interArea = max(0, xB - xA) * max(0, yB - yA) * max(0, zB - zA)
#
# boxAArea = (a[3] - a[0]) * (a[4] - a[1]) * (a[5] - a[2])
# boxBArea = (b[3] - b[0]) * (b[4] - b[1]) * (b[5] - b[2])
#
# iou = interArea / float(boxAArea + boxBArea - interArea)

In [2]:
def iou(a, b):
    zA = tf.maximum(a[0], b[0])
    yA = tf.maximum(a[1], b[1])
    xA = tf.maximum(a[2], b[2])
    zB = tf.minimum(a[3], b[3])
    yB = tf.minimum(a[4], b[4])
    xB = tf.minimum(a[5], b[5])

    interArea = tf.maximum(0, xB - xA) * tf.maximum(0, yB - yA) * tf.maximum(0, zB - zA)

    boxAArea = (a[3] - a[0]) * (a[4] - a[1]) * (a[5] - a[2])
    boxBArea = (b[3] - b[0]) * (b[4] - b[1]) * (b[5] - b[2])

    return interArea / (boxAArea + boxBArea - interArea)


In [3]:
import tensorflow as tf

a = tf.constant([0, 0, 1, 6, 7, 9])
b = tf.constant([2, 5, 3, 10, 10, 10])

iou = iou(a, b)

# init_op = tf.global_variables_initializer()
#
# with tf.Session() as sess:
#   sess.run(init_op)
#   iou_value = iou.eval()

In [4]:
tf.concat([a[:2], a[2:]], 0)

<tf.Tensor: id=139, shape=(6,), dtype=int32, numpy=array([0, 0, 1, 6, 7, 9], dtype=int32)>

In [5]:
import tensorflow as tf

boxes = tf.constant([
    [0, 0, 1, 6, 7, 9],
    [2, 5, 3, 10, 10, 10],
    [20, 50, 30, 100, 100, 100],
    [21, 40, 10, 330, 350, 360]
    ]
)

scores = tf.constant([
    0.2,
    0.4,
    0.1,
    0.6
])

boxes_prym = boxes[1:, :]

In [7]:
import sys
import numpy as np


def iou(a, b):
    zA = tf.maximum(a[0], b[0])
    yA = tf.maximum(a[1], b[1])
    xA = tf.maximum(a[2], b[2])
    zB = tf.minimum(a[3], b[3])
    yB = tf.minimum(a[4], b[4])
    xB = tf.minimum(a[5], b[5])

    interArea = tf.maximum(0, xB - xA) * tf.maximum(0, yB - yA) * tf.maximum(0, zB - zA)

    boxAArea = (a[3] - a[0]) * (a[4] - a[1]) * (a[5] - a[2])
    boxBArea = (b[3] - b[0]) * (b[4] - b[1]) * (b[5] - b[2])

    return interArea / (boxAArea + boxBArea - interArea)


def iou_evaluation(n, boxes, scores, highest_score_box, nms_threshold, index, removed_boxes):
    iou_value = iou(highest_score_box, boxes[index])

    (boxes, scores, index, removed_boxes) = tf.cond(
        tf.greater(iou_value, nms_threshold),
        lambda: (
            tf.concat([boxes[:index], boxes[index + 1:]], 0),
            tf.concat([scores[:index], scores[index + 1:]], 0),
            index,
            tf.add(removed_boxes, 1)
        ),
        lambda: (boxes, scores, tf.add(index, 1), removed_boxes)
    )
    return [tf.add(n, 1), boxes, scores, highest_score_box, nms_threshold, index, removed_boxes]


def nms_step(n, boxes, scores, nms_threshold, proposals):
    sorted_args = tf.argsort(scores, direction='DESCENDING')
    highest_score_arg = sorted_args[0]
    highest_score_box = boxes[highest_score_arg]
    number_of_processed = tf.constant(0)
    index = tf.constant(0)
    boxes = tf.concat([boxes[:highest_score_arg], boxes[highest_score_arg + 1:]], 0)
    scores = tf.concat([scores[:highest_score_arg], scores[highest_score_arg + 1:]], 0)
    boxes_size = tf.shape(boxes)[0]
    removed_boxes = tf.constant(0)

    (n_internal, boxes, scores, highest_score_box, nms_threshold, index, removed_boxes) = tf.while_loop(
        lambda n, b, s, h, t, i, r: n < boxes_size,
        lambda n, b, s, h, t, i, r: iou_evaluation(n, b, s, h, t, i, r),
        [
            number_of_processed,
            boxes,
            scores,
            highest_score_box,
            nms_threshold,
            index,
            removed_boxes
        ],
        shape_invariants=[
            number_of_processed.get_shape(),
            tf.TensorShape([None, 6]),
            tf.TensorShape([None]),
            tf.TensorShape([None]),
            nms_threshold.get_shape(),
            index.get_shape(),
            removed_boxes.get_shape()
        ]
    )
    return [
        tf.add(tf.add(n, 1), removed_boxes),
        boxes,
        scores,
        nms_threshold,
        tf.concat([proposals, [highest_score_arg]], 0)
    ]


def nms(boxes, scores, proposal_count=4, nms_threshold=0.5):
    number_of_processed = tf.constant(0)
    boxes_size = tf.shape(boxes)[0]
    nms_threshold = tf.constant(nms_threshold, dtype=tf.float64)

    proposals = tf.zeros([0])

    tf.print(boxes, output_stream=sys.stdout)

    number_of_processed, boxes, scores, nms_threshold, proposals = tf.while_loop(
        lambda n, b, s, t, p: tf.math.logical_and(n < boxes_size, tf.shape(p)[0] < proposal_count),
        lambda n, b, s, t, p: nms_step(n, b, s, t, p),
        [number_of_processed, boxes, scores, nms_threshold, proposals],
        shape_invariants=[
            number_of_processed.get_shape(),
            tf.TensorShape([None, 6]),
            tf.TensorShape([None]),
            nms_threshold.get_shape(),
            tf.TensorShape([None]),
        ]
    )

    return proposals[1:]

proposals = nms(boxes, scores)
# proposals = nms(boxes, scores).numpy()


[[0 0 1 6 7 9]
 [2 5 3 10 10 10]
 [20 50 30 100 100 100]
 [21 40 10 330 350 360]]


In [ ]:
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init_op)
  proposals_value = proposals.eval()

In [ ]:
a = [1, 2]
a[:1] + a[1 + 1:]

In [ ]:
boxes_size = tf.constant(4)
n = tf.constant(4)
